# Wisconsin

In [78]:
import datetime
import time
import wget
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import zipfile
from geopy.geocoders import Nominatim
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [2]:
ThisYear = str(datetime.datetime.now().year - 1)  + '-' + str(datetime.datetime.now().year)[2:]
PreString = 'https://dpi.wi.gov/sites/default/files/wise/downloads/enrollment_certified_'
PostString = '.zip'
File = PreString + ThisYear + PostString
File

'https://dpi.wi.gov/sites/default/files/wise/downloads/enrollment_certified_2017-18.zip'

In [3]:
ThisYearFile = wget.download(File)

100% [............................................................................] 749325 / 749325

In [4]:
LastYear = str(datetime.datetime.now().year - 2)  + '-' + str(datetime.datetime.now().year -1)[2:]
File = PreString + LastYear + PostString
LastYearFile = wget.download(File)

100% [............................................................................] 749421 / 749421

In [5]:
zf = zipfile.ZipFile(ThisYearFile) # having First.csv zipped file.
WIThYr = pd.read_csv(zf.open('enrollment_certified_' + ThisYear +'.csv'))

In [6]:
zf = zipfile.ZipFile(LastYearFile) # having First.csv zipped file.
WILsYr = pd.read_csv(zf.open('enrollment_certified_' + LastYear + '.csv'))

In [7]:
WILsYr.columns

Index(['SCHOOL_YEAR', 'AGENCY_TYPE', 'CESA', 'COUNTY', 'DISTRICT_CODE',
       'SCHOOL_CODE', 'GRADE_GROUP', 'CHARTER_IND', 'DISTRICT_NAME',
       'SCHOOL_NAME', 'GROUP_BY', 'GROUP_BY_VALUE', 'STUDENT_COUNT',
       'PERCENT_OF_GROUP'],
      dtype='object')

In [8]:
WILsYr = WILsYr[(WILsYr.GROUP_BY == 'Grade Level') & (WILsYr.GRADE_GROUP != '[All]') &
((WILsYr.GROUP_BY_VALUE == '11') | (WILsYr.GROUP_BY_VALUE == '12'))]

In [9]:
WIThYr = WIThYr[(WIThYr.GROUP_BY == 'Grade Level') & (WIThYr.GRADE_GROUP != '[All]') &
((WIThYr.GROUP_BY_VALUE == '11') | (WIThYr.GROUP_BY_VALUE == '12'))]

In [10]:
WILsYr.columns

Index(['SCHOOL_YEAR', 'AGENCY_TYPE', 'CESA', 'COUNTY', 'DISTRICT_CODE',
       'SCHOOL_CODE', 'GRADE_GROUP', 'CHARTER_IND', 'DISTRICT_NAME',
       'SCHOOL_NAME', 'GROUP_BY', 'GROUP_BY_VALUE', 'STUDENT_COUNT',
       'PERCENT_OF_GROUP'],
      dtype='object')

In [11]:
WILsYr.GROUP_BY_VALUE.drop_duplicates()

203    11
204    12
Name: GROUP_BY_VALUE, dtype: object

In [12]:
len(WILsYr[WILsYr['GROUP_BY_VALUE'] == '12'])

579

In [13]:
# Separate grades into different dfs for last year
WILsYrSr = WILsYr[WILsYr['GROUP_BY_VALUE'] == '12']
WIThYrSr = WIThYr[WIThYr['GROUP_BY_VALUE'] == '12']

In [14]:
# Separate grades into different dfs for this year
WILsYrJr = WILsYr[WILsYr['GROUP_BY_VALUE'] == '11']
WIThYrJr = WIThYr[WIThYr['GROUP_BY_VALUE'] == '11']


In [15]:
# Combine Jr and Sr as separate columns instead of rows
WIL = (pd.merge(WILsYrSr, WILsYrJr, how = 'inner', on = ['DISTRICT_CODE', 'SCHOOL_CODE']))
WIT = (pd.merge(WIThYrSr, WIThYrJr, how = 'inner', on = ['DISTRICT_CODE', 'SCHOOL_CODE']))

In [16]:
WIL = WIL.rename(columns = {'STUDENT_COUNT_x':'Juniors Last Year',
                       'STUDENT_COUNT_y': 'Seniors Last Year' })

In [17]:
WIT = WIT.rename(columns = {'STUDENT_COUNT_x':'Juniors This Year',
                       'STUDENT_COUNT_y': 'Seniors This Year' })

In [18]:
WI = pd.merge(WIL, WIT, how = 'inner', on = ['DISTRICT_CODE', 'SCHOOL_CODE'])

In [19]:
#WI['Junior Delta'] 

In [23]:
len(WI)

554

In [21]:
# Set the preferences for the firefox web browser
fp = webdriver.FirefoxProfile()
fp.set_preference('browser.download.folderList', 2)
fp.set_preference('browser.download.manager.showWhenStarting', False)
fp.set_preference('browser.download.dir', '/tmp')
fp.set_preference("http.response.timeout", 300)
fp.set_preference("dom.max_script_run_time", 300)
fp.set_preference('webdriver.load.strategy', 'unstable')
fp.set_preference('browser.helperApps.neverAsk.saveToDisk', 'text/plain, application/vnd.ms-excel, text/csv, text/comma-separated-values, application/octet-stream, application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
fp.update_preferences()

In [22]:
# MDE web app for hs enrollment data
searchURL = "https://apps4.dpi.wi.gov/SchoolDirectory/Search/PublicSchoolsSearch"
time.sleep(3)

# Launch web browser and navigate to the searchURL
driver = webdriver.Firefox(fp)
driver.get(searchURL)
time.sleep(3)

In [26]:
WI.head()

SCHOOL_YEAR_x_x AGENCY_TYPE_x_x  CESA_x_x COUNTY_x_x  DISTRICT_CODE  \
0         2016-17   Public school      10.0      Clark              7   
1         2016-17   Public school       5.0      Adams             14   
2         2016-17   Public school       2.0      Green             63   
3         2016-17   Public school       7.0   Kewaunee             70   
4         2016-17   Public school       4.0    Buffalo             84   

   SCHOOL_CODE GRADE_GROUP_x_x CHARTER_IND_x_x      DISTRICT_NAME_x_x  \
0         40.0     High School              No             Abbotsford   
1         40.0     High School              No  Adams-Friendship Area   
2         40.0     High School              No                 Albany   
3         40.0     High School              No                 Algoma   
4         40.0     High School              No                   Alma   

                 SCHOOL_NAME_x_x         ...          CESA_y_y COUNTY_y_y  \
0  Abbotsford Middle/Senior High         ...              10.0      Clark   
1          Adams-Friendship High         ...               5.0      Adams   
2                    Albany High         ...               2.0      Green   
3                    Algoma High         ...               7.0   Kewaunee   
4                      Alma High         ...               4.0    Buffalo   

  GRADE_GROUP_y_y CHARTER_IND_y_y      DISTRICT_NAME_y_y  \
0     High School              No             Abbotsford   
1     High School              No  Adams-Friendship Area   
2     High School              No                 Albany   
3     High School              No                 Algoma   
4     High School              No                   Alma   

                 SCHOOL_NAME_y_y  GROUP_BY_y_y GROUP_BY_VALUE_y_y  \
0  Abbotsford Middle/Senior High   Grade Level                 11   
1          Adams-Friendship High   Grade Level                 11   
2                    Albany High   Grade Level                 11   
3                    Algoma High   Grade Level                 11   
4                      Alma High   Grade Level                 11   

  Seniors This Year PERCENT_OF_GROUP_y_y  
0                58                15.68  
1               135                29.03  
2                24                25.00  
3                59                18.15  
4                21                30.00  

[5 rows x 50 columns]

In [84]:
Search = driver.find_element_by_id("txtSearch")
Search.send_keys("Abbotsford Middle/Senior High")
button = driver.find_element_by_xpath("//input[@value='Search']")

time.sleep(1)
time.sleep(1)
ActionChains(driver).move_to_element(button).click().perform()
time.sleep(1)
